An attempt to understand polypcolor and mosaic

In [1]:
#Packages 
import numpy as np
import xarray as xr
import cmocean.cm as cmo
import matplotlib.pyplot as plt
import mpas_tools
from mpas_tools.planar_hex import make_planar_hex_mesh
from mpas_tools.mesh.conversion import convert, cull, mask
from mpas_tools.io import write_netcdf
import mosaic
from dask.diagnostics import ProgressBar

In [4]:
# open mesh
mesh_path = '/global/cfs/projectdirs/e3sm/inputdata/ocn/mpas-o/EC30to60E2r2/ocean.EC30to60E2r2.210210.nc'
mesh = xr.open_dataset(mesh_path)
dsMesh = mesh

# open mask
wmtnb_filepath = '/global/u2/k/kuyeda/WMT_NBs/'
mask = xr.open_dataset(wmtnb_filepath + 'LS_CEV_masks.nc')
mask

# open simulation data
bmm_filepath = '/pscratch/sd/b/bmoorema/run_001_062/'
ds = xr.open_dataset(bmm_filepath + '20210421_sim7_CORE_60to30E2r2.mpaso.hist.am.timeSeriesStatsMonthly.0063-12-01.nc')
nVertLevels_ind = 30
nVertLevelsP1_ind = 30

# select datavariables
dvs = ['timeMonthly_avg_activeTracers_temperature','timeMonthly_avg_activeTracers_salinity','timeMonthly_avg_latentHeatFlux',
               'timeMonthly_avg_sensibleHeatFlux','timeMonthly_avg_velocityMeridional','timeMonthly_avg_velocityZonal','timeMonthly_avg_vertVelocityTop','timeMonthly_avg_normalVelocity']
dsSims = ds[dvs].isel(Time=0,nVertLevels=nVertLevels_ind,nVertLevelsP1=nVertLevels_ind)



/tmp/ipykernel_854183/117611068.py:13: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset(bmm_filepath + '20210421_sim7_CORE_60to30E2r2.mpaso.hist.am.timeSeriesStatsMonthly.0063-12-01.nc')


In [7]:
mesh.latCell

<xarray.DataArray 'latCell' (nCells: 236853)> Size: 2MB
array([ 1.56982964, -0.46330786, -0.46346924, ..., -0.20522028,
        0.44590281,  0.14292051], shape=(236853,))
Dimensions without coordinates: nCells
Attributes:
    units:      radians
    long_name:  Latitude location of cell centers in radians.

In [5]:
# set up mesh to apply polypcolor
mesh.attrs['is_periodic'] = 'YES'
mesh.attrs['x_period'] = 360
mesh.attrs['y_period'] = 0.0

# Mosaic descriptor object, needed to plot the polygons
descriptor = mosaic.Descriptor(mesh,use_latlon=False)

In [4]:
def polypcolor_plots(ds_str, dsSims, descriptor):
    # get the correct dataset
    ds_name = ds_str
    ds = dsSims[ds_str]
    
    # use a .where to convert the lowest value (~-10 * 10^33) to a nan
    ds_filled_nans = ds.where(ds != ds.min())
    
    # create a new colorbar centered at 0
    # create a new colorbar 
    # bounds
    ds_min = float(ds_filled_nans.min().values)
    ds_max = float(ds_filled_nans.max().values)
    
    if (ds_min < 0) and (ds_max > 0):
        pivot = 0
    else:
        pivot = np.round(((ds_max - ds_min) / 2) + ds_min)
    
    new_cmap = cmo.tools.crop(cmo.balance,ds_min,ds_max,pivot)
    
    # plot
    fig,ax = plt.subplots(nrows=1,ncols=1,constrained_layout=True)
    
    c = mosaic.polypcolor(ax,descriptor,
                      ds_filled_nans,cmap=new_cmap,
                      antialiaseds=False)
    fig.colorbar(c, extend='both')
    ax.set_xlim(0,360)
    ax.set_title(ds_name + '\n Level=30, time=0')
    
    # # save figs 
    # fig_path = '/global/homes/k/kuyeda/WMT_NBs/initial_figs/polypcolor_figs_masked/'
    # plt.savefig(fig_path + ds_name + '.png' , bbox_inches='tight')
    # plt.close(fig)

In [29]:
def polypcolor_masked_plots(ds_str, dsSims, mask, descriptor):
    # establish masks
    cellmask = mask.regionCellMasks.isel(nRegions=0).astype(bool)
    edgemask = mask.regionEdgeMasks.isel(nRegions=0).astype(bool)
    vertexmask = mask.regionVertexMasks.isel(nRegions=0).astype(bool)
    
    # get the correct dataset
    ds_name = ds_str
    ds = dsSims[ds_str]
    # create nans
    ds_nans = ds.where(ds != ds.min())
    
    # specify which mask based on coordinates of datavariable
    if 'nCells' in ds_nans.dims:
        grid_mask = cellmask
    elif 'nEdges' in ds_nans.dims:
        grid_mask = edgemask
    
    
    # apply the mask
    ds_masked = ds_nans.where(grid_mask,drop=False)

    # create a new colorbar centered at 0
    # create a new colorbar 
    # bounds
    ds_min = float(ds_masked.min().values)
    ds_max = float(ds_masked.max().values)
    
    if (ds_min < 0) and (ds_max > 0):
        pivot = 0
    else:
        pivot = np.round(((ds_max - ds_min) / 2) + ds_min, decimals=5)
    
    new_cmap = cmo.tools.crop(cmo.balance,ds_min,ds_max,pivot)
    
    # plot
    fig,ax = plt.subplots(nrows=1,ncols=1,constrained_layout=True)
    
    c = mosaic.polypcolor(ax,descriptor,
                      ds_masked,cmap=new_cmap,
                      antialiaseds=False)
    fig.colorbar(c, extend='both')
    ax.set_xlim(290, 315)
    ax.set_ylim(51.5, 68)
    ax.set_title(ds_name + '\n Level=30, time=0')
    
    # # save figs 
    # fig_path = '/global/homes/k/kuyeda/WMT_NBs/initial_figs/polypcolor_figs/'
    # plt.savefig(fig_path + 'LSmask_' + ds_name + '.png' , bbox_inches='tight')
    # plt.close(fig)

In [81]:
for i in range(0,len(dvs)):
    polypcolor_plots(dvs[i],dsSims,descriptor)

In [30]:
for i in range(0,len(dvs)):
    polypcolor_masked_plots(dvs[i],dsSims,mask,descriptor)